In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 32744498
paper_name = 'ivashov_teis_2020' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [7]:
original_data = pd.read_excel('raw_data/elife-58246-supp2-v3.xlsx', sheet_name='Table S2', skiprows=7)

In [8]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 129 x 6


In [9]:
original_data.head()

,Systematic name,Standard name,- N (% cells with Mup1-pH signal),- N phenotype,+ Met (% cells with Mup1-pH signal),SGD annotated function
0,YIL087C,AIM19,79.0,1,12,"protein of unknown function, mitochondrial pro..."
1,YIL158W,AIM20,77.0,1,18,"protein of unknown function, GFP-fusion protei..."
2,YHR126C,ANS1,100.0,1,1,"GPI protein, GFP and mCherry fusion proteins l..."
3,YHL019C,APM2,100.0,1,2,homolog of the medium chain of the clathrin-as...
4,YHR018C,ARG4,100.0,1,55,involved in biosynthesis of arginine


In [10]:
original_data['orf'] = original_data['Systematic name'].astype(str)

In [11]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [12]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

            Systematic name Standard name  - N (% cells with Mup1-pH signal)  \
index_input                                                                    
128                     NaN           NaN                                NaN   

            - N phenotype + Met (% cells with Mup1-pH signal)  \
index_input                                                     
128                                                       NaN   

            SGD annotated function  orf  
index_input                              
128                            NaN  NAN  


In [14]:
original_data = original_data.loc[t,:]

In [15]:
original_data['data'] = pd.to_numeric(original_data['- N (% cells with Mup1-pH signal)'], errors='coerce')

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data['data'] = -original_data['data']/100

In [21]:
original_data.shape

(128, 1)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [21838]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,21838
data_type,value
orf,
YAL031C,-0.48
YBL101C,-0.88
YBR020W,-0.80
YBR063C,-0.56
YBR129C,-0.48


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 1


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21838
,data_type,value
gene_id,orf,
29,YAL031C,-0.48
189,YBL101C,-0.88
216,YBR020W,-0.80
259,YBR063C,-0.56
323,YBR129C,-0.48


# Normalize

In [29]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [30]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [31]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      21838          
data_type       value    valuez
gene_id orf                    
29      YAL031C -0.48 -3.519979
189     YBL101C -0.88 -6.453295
216     YBR020W -0.80 -5.866632
259     YBR063C -0.56 -4.106642
323     YBR129C -0.48 -3.519979

# Print out

In [32]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [33]:
from IO.save_data_to_db3 import *

In [34]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 32744498...
Inserting the new data...


100%|██████████| 1/1 [00:00<00:00,  3.04it/s]

Updating the data_modified_on field...
